In [ ]:
# use when recorded both PMT channels, and want to suite2p both channels

In [ ]:
# all channels go as input to suite2p
import os
import suite2p
from suite2p.run_s2p import run_s2p
import json
import sys
import numpy as np
import time
import gc
from pathlib import Path
from glob import glob
from ScanImageTiffReader import ScanImageTiffReader
import h5py
from scipy.io import savemat
import multiprocessing
import logging

def tiffs2array_meso(movie_list, x_slice, y_slice, t_slice,planes,channels, opsjson):
    data = [LoadTif(str(mov),planes,channels) for mov in movie_list] #CAREFUL HERE should slice after concatenation for Satsuma RF, but NOT for Frankenrig

    data = np.concatenate(data)
    print('concatenated data from all tiffs, total size: ' + str(data.shape))
    
    print('number of frames: ' + str(data.shape[0]))
    print('opsjson[''lines''] shape: ' + str( np.shape(opsjson['lines']) ))
    print('opsjson[''dx'']: ' + str(opsjson['dx']))
    print(data.shape[0], np.shape(opsjson['lines'])[1], np.shape(opsjson['lines'])[0]*opsjson['dx'][1] )

    ylen = np.shape(opsjson['lines'])[1]
    xlen = np.shape(opsjson['lines'])[0]*opsjson['dx'][1]
    xbounds = opsjson['dx'].copy()
    xbounds.append(xlen)
    datars = np.zeros((data.shape[0], ylen, xlen ), dtype=np.int16)
    for istrip in range(np.shape(opsjson['lines'])[0]):
        x_strip= slice(xbounds[istrip],xbounds[istrip+1])
        y_strip= slice(0,ylen)
        datars[t_slice,y_strip,x_strip] = data[:,opsjson['lines'][istrip],:].copy()

    print('reshaped data size: ' + str(datars.shape))
    data = datars[t_slice, y_slice, x_slice] #.copy()
    return data

def LoadTif(mov_path,planes,channels):
    with ScanImageTiffReader(mov_path) as reader:
        data = reader.data()
    Tend = int( np.floor(data.shape[0]/(planes*channels)) * (planes*channels) )
    t_slice = slice(0,Tend, 1)
    data = data[t_slice,:,:]
    return data

def crop_and_save_h5_meso(movs,tiff_folder,out_path,expno, x_slice,y_slice,planes,channels, opsjson):
    # takes a list of tifs. Concatenates each plane, then slices and crops the big movie, then saves it as .h5 file in a folder inside tiff_folder\
    # process multiple folders
    # instead of making one h5 per plane, just make it into one big h5
    # movs: list of tifs to be processed
    # tif folder: directory where the result h5 file will be saved (inside the corresponding plane folder)
    # x_slice: cropping of the FOV in x
    # x_slice: cropping of the FOV in y
    # planes: number of planes for each tif file
    # channels: number of channels"""
    tic = time.time()

    #exp_name1 = exp_name.replace("//","")
    outname = [0]*planes
    outdir=[0]*planes
    try:
        os.mkdir(out_path)
    except:
        if os.path.isdir(out_path):
            print('directory already exists')
        else:
            print('Cannot create directory.. probably the name is wrong')
            print(out_path)
            

    t_slice = slice(0, None, 1)

    cropped_mov = tiffs2array_meso(movs,x_slice,y_slice,t_slice,planes,channels, opsjson)
    print(cropped_mov.shape)
    
    plane = 0
#     #for plane in np.arange(0,planes):
#     t_slice = slice(plane*channels, cropped_mov.shape[0], channels * planes) 
#     x_slice = slice(0,cropped_mov.shape[2], 1)
#     y_slice = slice(0,cropped_mov.shape[1], 1)
    t_slice = slice(0,None, 1)
    x_slice = slice(0,None, 1) 
    y_slice = slice(0,None, 1)
    outdir[plane] = out_path #+'plane_'+ str(plane) + '//'
    try:
        os.mkdir(outdir[plane])
    except:
        print('couldnt make directory')
        print(str(outdir[plane]))

    outname[plane]=outdir[plane] + 'cropped_mov_' + str(expno) + '.h5'
    print(f'now processing {outname[plane]}')
    # try:
    hf = h5py.File(outname[plane], 'a')
    cropped_mov1 = cropped_mov[t_slice, y_slice, x_slice]
    print(cropped_mov1.shape)
    hf.create_dataset('data', data=cropped_mov1, dtype= 'uint16')
    hf.close()
    print('done saving ' + str(outname[plane]))
    # delete the array and clean RAM
    del cropped_mov1
    del cropped_mov
    gc.collect()
    # except:
    #     print(f'the file {outname[plane]} already exists')

    toc = time.time() - tic
    print('All saved,took  ' + str(toc) + 'secs')

    return


In [ ]:
tic1 = time.time()

# INFO ABOUT YOUR FILES
baseFreq = 3.00
channels = 2
planes = 1

# HOW TO FIND COORDINATES CORRESPONDING TO ActualHoloFOV  BOUNDARIES IN YOUR CURRENT MROI SETTING
# % find out ActualHoloFOV boundaries using C:\Users\MesoDAQ\Documents\MATLAB\MesoSICode\HScode\suite2p_pipeline\convertcoords_HoloFOVtoCurrentFOV.m
# % run the following in the MATLAB instance running Scanimage
# % the arguments for convertcoords_HoloFOVtoCurrentFOV should already be in the workspace if C:\Users\MesoDAQ\Documents\MATLAB\MesoSICode\makeMasks3D_holeburn.m was run that day
# % xynew(:,1) is the horizontal axis, xynew(:,2) is the vertical axis
# % set xlb, xub (horizontal axis) and ylb, yub (vertical axis) accordingly in mesocroph5.ipynb 
# xyorig = [50 50; fullnpix_orig(1)-50 fullnpix_orig(2)-50];
# xynew = convertcoords_HoloFOVtoCurrentFOV(hSI,xyorig, fullnpix_orig, fullxsize_orig, fullysize_orig, fullxcenter_orig, fullycenter_orig)
# xynew =
#          200          67
#         1304         617
x_start = 200; # horizontal
x_end = 1304; # horizontal
y_start = 67;
y_end = 617;

# drive = 'M://'
# user_name = 'ICmesoholoexpts_scanimage'
drive = 'D://'
user_name = 'HS'
mouse = 'HS_Chrome2f_3'
date = '220922'
expList = ['retinotopy0', 'staticICtxi0', 'staticgratings']

#where your tifs are
tiffFolderList = [drive+ user_name + '//' + mouse +'//' + date + '//' + exp_name + '//' for exp_name in expList]
#where you want to save the output hdf5 file
suffix = 'ClosedLoop_allch//' #'suite2pAnalysis//'
data_path = drive+ user_name + '//' + mouse +'//' + date + '//'
out_path = drive+ user_name + '//' + mouse +'//' + date + '//' + suffix

In [ ]:
# only took 130 sec for singl
x_slice= slice(x_start-1,x_end)
y_slice= slice(y_start-1,y_end)
tic1 = time.time()

os.chdir(data_path)
with open('ops.json') as f:
    opsjson = json.load(f)
print(opsjson.keys())

gc.collect()
for exp_name, tiff_folder in zip(expList,tiffFolderList):
    gc.collect()
    pth = Path(tiff_folder)
    movs = list(pth.glob('*.tif'))
    crop_and_save_h5_meso(movs,tiff_folder,out_path,exp_name, x_slice,y_slice,planes,channels, opsjson)

toc1 = time.time()-tic1
print(toc1)
print('completed h5 conversion')


In [ ]:
savemat(out_path+'online_params.mat', {'expList':expList, 'numchannels':channels, 'Nplanes':planes, 'x_start':x_start, 'x_end':x_end, 'y_start':y_start, 'y_end':y_end, 'numtimepointstiffile':numtimepointstiffileagg})

In [ ]:
# # troubleshooting rewriting tiffs2array_meso
# os.chdir(data_path)
# with open('ops.json') as f:
#     opsjson = json.load(f)
# print(opsjson.keys())

# tiff_folder = 'D://HS//HS_Ai203_2//220531//retinotopy2//'
# pth = Path(tiff_folder)
# movie_list = list(pth.glob('*.tif'))
# # movs[:10]
# data = [LoadTif(str(mov),planes,channels) for mov in movie_list] #CAREFUL HERE should slice after concatenation for Satsuma RF, but NOT for Frankenrig
# data = np.concatenate(data)
# print('concatenated data from all tiffs, total size: ' + str(data.shape))
# print('number of frames: ' + str(data.shape[0]))
# print('opsjson[''lines''] shape: ' + str( np.shape(opsjson['lines']) ))
# print('opsjson[''dx'']: ' + str(opsjson['dx']))
# print(data.shape[0], np.shape(opsjson['lines'])[1], np.shape(opsjson['lines'])[0]*opsjson['dx'][1] )

# # # Matlab code for reshaping the data
# # # in Matlab, tif data size 608 7620 numframes, and reshaped data size is 1484 3040 numframes
# # % reshape the data
# # tempdata = zeros(max(Ly), sum(Lx), size(temp,3), 'int16');
# # for istrip = 1:numel(Ly)
# #     tempdata(1:Ly(istrip), sum(Lx(1:istrip-1))+1:sum(Lx(1:istrip)), :) = permute(temp(:,data.lines{istrip}+1,:),[2 1 3]);
# # end
# # % crop the data
# # temp = tempdata(ylb:yub, xlb:xub, :);
# x_slice= slice(x_start,x_end+1)
# y_slice= slice(y_start,y_end+1)

# tic1 = time.time()
# ylen = np.shape(opsjson['lines'])[1]
# xlen = np.shape(opsjson['lines'])[0]*opsjson['dx'][1]
# xbounds = opsjson['dx'].copy()
# xbounds.append(xlen)
# datars = np.zeros((data.shape[0], ylen, xlen ))
# for istrip in range(np.shape(opsjson['lines'])[0]):
#     x_strip= slice(xbounds[istrip],xbounds[istrip+1])
#     y_strip= slice(0,ylen)
#     print(data[:,opsjson['lines'][istrip],:].shape)
#     datars[:,y_strip,x_strip] = data[:,opsjson['lines'][istrip],:].copy()

# data = datars[:, y_slice, x_slice].copy()
# print('reshaped data size: ' + str(datars.shape))
# print('cropped data size: ' + str(data.shape))

# toc1 = time.time()-tic1
# print(toc1)

In [ ]:
# or if you want to play with the parameters define everything here:
# Should comment what everything is, not sure about all the parameters.
ops = {'look_one_level_down': 0.0,
 'fast_disk': [],
 'delete_bin': True,
 'mesoscan': False,
 'bruker': False,
 'h5py': [],
 'h5py_key': 'data',
 'save_path0': [],
 'save_folder': [],
 'subfolders': [],
 'move_bin': False,
 'nplanes': planes,
 'nchannels': channels,
 'functional_chan': 1,
 'tau': 1.5,
 'fs': baseFreq/planes,
 'force_sktiff': False,
 'frames_include': -1,
 'multiplane_parallel': False,
 'preclassify': 0.0,
 'save_mat': True,
 'save_NWB': False,
 'combined': 1.0,
 'aspect': 2.0,
 'do_bidiphase': True,
 'bidiphase': 0,
#  'bidi_corrected': False,
 'do_registration': 1,
 'two_step_registration': False,
 'keep_movie_raw': False, # if two_step_registration = 1 this has to also be true.
 'nimg_init': 1000,
 'batch_size': 2000,
 'maxregshift': 0.1,
 'align_by_chan': 1,
 'reg_tif': False,
 'reg_tif_chan2': False,
 'subpixel': 10,
 'smooth_sigma_time': 1.0, # default 0
 'smooth_sigma': 1.15, # pixels, default 1.15
 'th_badframes': 1.0,
 'pad_fft': False,
 'nonrigid': False,
 'block_size': [128, 128],
 'snr_thresh': 1.2,
 'maxregshiftNR': 5.0,
 '1Preg': False,
 'spatial_hp': 25,
 'spatial_hp_reg': 26.0,
 'spatial_hp_detect': 25,
 'pre_smooth': 0,
 'spatial_taper': 50.0,
 'roidetect': True,
 'spikedetect': True,
 'sparse_mode': False,
 'diameter': 9,
 'spatial_scale': 1, # default 0, if set to 0, then the algorithm determines it automatically (recommend this on the first try)
 'connected': True,
 'nbinned': 5000, # maximum number of binned frames to use for ROI detection
 'max_iterations': 20, # default 20. how many iterations over which to extract cells - at most ops[‘max_iterations’], 
                        #but usually stops before due to ops[‘threshold_scaling’] criterion
 'threshold_scaling': 2.0, # default: 5.0. this controls the threshold at which to detect ROIs.. if you set this higher, then fewer ROIs will be detected
 'max_overlap': 0.5, # default: 0.75
 'high_pass': 100, #  default: 100. time window size
 'inner_neuropil_radius': 2, # default: 2. number of pixels to keep between ROI and neuropil donut
 'min_neuropil_pixels': 350, #default: 350. minimum number of pixels used to compute neuropil for each cell
 'allow_overlap': False, #default: False. whether or not to extract signals from pixels which belong to two ROIs.
 'chan2_thres': 0.65,
 'baseline': 'maximin',
 'win_baseline': 60.0,
 'sig_baseline': 10.0,
 'prctile_baseline': 8.0,
 'neucoeff': 0.7}

In [ ]:
# when running only one plane, takes the same amount of time (12 min in benchmark data)
tic2 = time.time()
ops['data_path'] = [out_path]
ops['h5py'] = out_path
ops1 = run_s2p(ops = ops, db = {})
toc2 = time.time() - tic2
print('Suite2p took ' + str(toc2/60) + 'mins')

In [ ]:
# # suite2p took 681s, or 12 min
# #list of files to analyze:
# outname = [0]*planes
# outdir=[0]*planes 

# #save output data
# for i in range(planes):
#     outdir[i]= out_path + 'plane_'+str(i)+'//'
#     outname[i]= out_path + 'plane_'+str(i)+'//'

# print(outdir)
# print(outname[0])

# # prepare the processes for each plane
# db = []
# jobs = []
# if __name__ == '__main__':
    
#     for i in range(planes):
#     #while the # of planes is less than the max number of cores we want to use (leaving 2 cores for scanimage)
#        if i<os.cpu_count()-2:
#     # Define the dataset
#             this_db = {
#               'h5py':outname[i], # a single h5 file path[p]
#               'h5py_key': ['data'],
#               'look_one_level_down': True, # whether to look in ALL subfolders when searching for tiffs,
#               'save_path0': outdir[i],
#               'data_path':  [], # a list of folders with tiffs
#                                                      # (or folder of folders with tiffs if look_one_level_down is True, or subfolders is not empty)
#               'subfolders': [], # choose subfolders of 'data_path' to look in (optional)
#               'fast_disk': [] }# string which specifies where the binary file will be stored (should be an SSD)      
#             print(f'Will be processing this data : {this_db}')
#             db.append(this_db)
#             p = multiprocessing.Process(target=run_s2p, args=(ops,this_db))
#             p.start()          
#             jobs.append(p)
#        else:
#             print(f'Hey, do you really want to run all these {i} cores at the same time?')
            
# #run each plane in parallel
# logging.basicConfig(level=logging.INFO)
# logging.info('Starting suite2p parallel processing in a different CPU core for each plane')

# tic = time.time()
# if len(jobs)<os.cpu_count()-2:
#     for job in jobs:
#         #print ('Parallel processing:  ' + str(outname[i]))
#         job.join()
#     toc = time.time() - tic
#     print('All saved,took  ' + str(toc) + 'secs')
# else:
#    print(f'Hey, do you really want to run all these {i} cores at the same time?')

# toc2 = time.time() - tic1
# print('Preprocessing took ' + str(toc2/60) + 'mins')